<a href="https://colab.research.google.com/github/AkankshaB123/leet-code/blob/main/Continuous_product_growth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***1.) reg. Learning 1: - *1.0 when division is not working despite denominator being in decent state(NULL IF - 0 or coalesce)***
# ***2.) reg. Learning 2: Use of Min(growth rate)>0 logic for continuous growth***

In [2]:
import sqlite3
import pandas as pd

# Create SQLite in-memory DB
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

In [3]:
# Create txn_data table
cursor.execute("""
CREATE TABLE txn_data (
    month_year TEXT,
    brand TEXT,
    product TEXT,
    category TEXT,
    sales NUMERIC
);
""")

# Sample data
data = [
    ("2023-01-01", "BrandA", "Prod1", "Cat1", 100),
    ("2023-02-01", "BrandA", "Prod1", "Cat1", 120),
    ("2023-03-01", "BrandA", "Prod1", "Cat1", 150),

    ("2023-01-01", "BrandA", "Prod2", "Cat1", 200),
    ("2023-02-01", "BrandA", "Prod2", "Cat1", 180),  # negative growth
    ("2023-03-01", "BrandA", "Prod2", "Cat1", 190),

    ("2023-01-01", "BrandB", "Prod3", "Cat2", 50),
    ("2023-02-01", "BrandB", "Prod3", "Cat2", 60),
    ("2023-03-01", "BrandB", "Prod3", "Cat2", 72)
]

cursor.executemany("INSERT INTO txn_data VALUES (?,?,?,?,?)", data)
conn.commit()

In [4]:
query_cte1 = """
WITH revised_txn_data AS (
    SELECT
        month_year,
        brand,
        product,
        category,
        sales,
        LAG(sales) OVER (
            PARTITION BY brand, product, category
            ORDER BY month_year
        ) AS prev_sales
    FROM txn_data
)
SELECT * FROM revised_txn_data
ORDER BY brand, product, month_year;
"""

df_cte1 = pd.read_sql_query(query_cte1, conn)
df_cte1

,month_year,brand,product,category,sales,prev_sales
0,2023-01-01,BrandA,Prod1,Cat1,100,NaN
1,2023-02-01,BrandA,Prod1,Cat1,120,100.0
2,2023-03-01,BrandA,Prod1,Cat1,150,120.0
3,2023-01-01,BrandA,Prod2,Cat1,200,NaN
4,2023-02-01,BrandA,Prod2,Cat1,180,200.0
5,2023-03-01,BrandA,Prod2,Cat1,190,180.0
6,2023-01-01,BrandB,Prod3,Cat2,50,NaN
7,2023-02-01,BrandB,Prod3,Cat2,60,50.0
8,2023-03-01,BrandB,Prod3,Cat2,72,60.0


In [21]:
query_cte2_fixed = """
WITH revised_txn_data AS (
    SELECT
        month_year,
        brand,
        product,
        category,
        sales,
        LAG(sales) OVER (
            PARTITION BY brand, product, category
            ORDER BY month_year
        ) AS prev_sales
    FROM txn_data
)
SELECT
    month_year,
    brand,
    product,
    category,
    sales,
    prev_sales,
    sales - prev_sales AS diff,
    (sales - prev_sales) * 1.0 / prev_sales AS ratio,
    ROUND((sales - prev_sales) * 1.0 / prev_sales * 100.0, 2) AS growth_rate_pct
FROM revised_txn_data
ORDER BY brand, product, month_year;
"""

df_cte2_fixed = pd.read_sql_query(query_cte2_fixed, conn)
df_cte2_fixed


,month_year,brand,product,category,sales,prev_sales,diff,ratio,growth_rate_pct
0,2023-01-01,BrandA,Prod1,Cat1,100,NaN,NaN,NaN,NaN
1,2023-02-01,BrandA,Prod1,Cat1,120,100.0,20.0,0.200000,20.00
2,2023-03-01,BrandA,Prod1,Cat1,150,120.0,30.0,0.250000,25.00
3,2023-01-01,BrandA,Prod2,Cat1,200,NaN,NaN,NaN,NaN
4,2023-02-01,BrandA,Prod2,Cat1,180,200.0,-20.0,-0.100000,-10.00
5,2023-03-01,BrandA,Prod2,Cat1,190,180.0,10.0,0.055556,5.56
6,2023-01-01,BrandB,Prod3,Cat2,50,NaN,NaN,NaN,NaN
7,2023-02-01,BrandB,Prod3,Cat2,60,50.0,10.0,0.200000,20.00
8,2023-03-01,BrandB,Prod3,Cat2,72,60.0,12.0,0.200000,20.00


In [22]:
query_continuous = """
WITH revised_txn_data AS (
    SELECT
        month_year, brand, product, category, sales,
        LAG(sales) OVER (
            PARTITION BY brand, product, category
            ORDER BY month_year
        ) AS prev_sales
    FROM txn_data
),
growth_rate_computation AS (
    SELECT
        month_year, brand, product, category, sales, prev_sales,
        CASE WHEN prev_sales IS NULL THEN NULL
             ELSE ROUND((sales - prev_sales)*1.0 / NULLIF(prev_sales,0) * 100.0, 2) END AS growth_rate_pct
    FROM revised_txn_data
)
SELECT
    brand,
    category,
    product
FROM growth_rate_computation
GROUP BY brand, category, product
HAVING    -- at least one observed period
MIN(growth_rate_pct) > 0;    -- all observed periods positive
"""

df_continuous = pd.read_sql_query(query_continuous, conn)
df_continuous

,brand,category,product
0,BrandA,Cat1,Prod1
1,BrandB,Cat2,Prod3


In [23]:
query_full = """
WITH revised_txn_data AS (
    SELECT month_year, brand, product, category, sales,
           LAG(sales) OVER (PARTITION BY brand, product, category ORDER BY month_year) AS prev_sales
    FROM txn_data
),
growth_rate_computation AS (
    SELECT month_year, brand, product, category, sales, prev_sales,
           CASE WHEN prev_sales IS NULL THEN NULL
                ELSE ROUND((sales - prev_sales)*1.0 / NULLIF(prev_sales,0) * 100.0, 2) END AS growth_rate_pct
    FROM revised_txn_data
),
product_continuity AS (
    SELECT brand, category, product,
           CASE WHEN COUNT(growth_rate_pct)=0 THEN 'only_one_period'
                WHEN MIN(growth_rate_pct) > 0 THEN 'continuous_positive_growth'
                ELSE 'not_continuous' END AS continuity_status
    FROM growth_rate_computation
    GROUP BY brand, category, product
)
SELECT g.month_year, g.brand, g.category, g.product, g.sales, g.prev_sales, g.growth_rate_pct, p.continuity_status
FROM growth_rate_computation g
JOIN product_continuity p
  ON g.brand = p.brand AND g.category = p.category AND g.product = p.product
ORDER BY g.brand, g.product, g.month_year;
"""

df_full = pd.read_sql_query(query_full, conn)
df_full

,month_year,brand,category,product,sales,prev_sales,growth_rate_pct,continuity_status
0,2023-01-01,BrandA,Cat1,Prod1,100,NaN,NaN,continuous_positive_growth
1,2023-02-01,BrandA,Cat1,Prod1,120,100.0,20.00,continuous_positive_growth
2,2023-03-01,BrandA,Cat1,Prod1,150,120.0,25.00,continuous_positive_growth
3,2023-01-01,BrandA,Cat1,Prod2,200,NaN,NaN,not_continuous
4,2023-02-01,BrandA,Cat1,Prod2,180,200.0,-10.00,not_continuous
5,2023-03-01,BrandA,Cat1,Prod2,190,180.0,5.56,not_continuous
6,2023-01-01,BrandB,Cat2,Prod3,50,NaN,NaN,continuous_positive_growth
7,2023-02-01,BrandB,Cat2,Prod3,60,50.0,20.00,continuous_positive_growth
8,2023-03-01,BrandB,Cat2,Prod3,72,60.0,20.00,continuous_positive_growth
